In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os

import statsmodels.formula.api as smf


# Load data

In [3]:
data_path = "feature_with_ecg_df_test_lead2.csv"
data_df = pd.read_csv(data_path)
data_df

,cycle_ID,patient_ID,interval_ID,block_ID,channel_ID,segment_ID,start_ID,end_ID,r_ID,r_t,...,ecg295,ecg296,ecg297,ecg298,ecg299,ecg300,r_ID_abs,r_ID_abs_ref,selected,duration
0,1,771495,8748755,1,2,1,1,66705,820,2015-12-22 23:12:04.099549952,...,1.290715,1.403295,1.606575,1.857944,2.039397,2.014317,820,820,True,277.528
1,2,771495,8748755,1,2,1,1,66705,907,2015-12-22 23:12:04.461459968,...,1.152530,1.248552,1.434638,1.692337,1.924521,1.995100,907,907,True,277.528
2,3,771495,8748755,1,2,1,1,66705,994,2015-12-22 23:12:04.823380224,...,0.954466,1.053145,1.247815,1.520770,1.775172,1.874207,994,994,True,277.528
3,4,771495,8748755,1,2,1,1,66705,1085,2015-12-22 23:12:05.201929984,...,1.468770,1.449762,1.533679,1.723174,1.924213,1.983855,1085,1085,True,277.528
4,5,771495,8748755,1,2,1,1,66705,1167,2015-12-22 23:12:05.543040000,...,1.150959,1.294285,1.518599,1.771597,1.948649,1.938984,1167,1167,True,277.528
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114407,308,536886,8748726,1,2,2,4442,31910,27061,2015-01-06 01:31:59.848000000,...,0.521465,0.680327,0.817151,0.849854,0.728921,0.478241,31502,31502,True,132.808
114408,309,536886,8748726,1,2,2,4442,31910,27157,2015-01-06 01:32:00.247380224,...,0.482723,0.645244,0.786819,0.825585,0.711836,0.468551,31598,31598,True,132.808
114409,310,536886,8748726,1,2,2,4442,31910,27253,2015-01-06 01:32:00.646749952,...,0.497711,0.666044,0.814049,0.855897,0.739084,0.487000,31694,31694,True,132.808
114410,311,536886,8748726,1,2,2,4442,31910,27349,2015-01-06 01:32:01.046129920,...,0.541680,0.717542,0.873392,0.916993,0.791512,0.521507,31790,31790,True,132.808


# Load Model

In [5]:
from src.utils.ECG_data_loading import *

In [31]:
from torch.utils.data import DataLoader
from src.methods.supervised_1d import SupervisedModel_1D
from src.models.CLOCS_1D import cnn_network_contrastive
import torch.nn as nn
from pathlib import Path
from src.utils.metrics import accuracy_at_k, weighted_mean, AUROC
import time

In [14]:
model = cnn_network_contrastive(zero_init_residual=None, embedding_dim=16,
                   stride=2, c4_multiplier=3)
# remove fc layer
model.fc = nn.Identity()

2 0.1 0.1 0.1 1 16 CLOCS 


In [34]:
# build paths
ckpt_dir = Path("trained_models/linear/wxjvsino")
# ckpt_dir = Path("trained_models/linear/wxjvsino/None-wxjvsino-epoch=21-val_auroc=0.9236.ckpt")
args_path = ckpt_dir / "args.json"
ckpt_path = [ckpt_dir / ckpt for ckpt in os.listdir(ckpt_dir) if ckpt.endswith("val_auroc=0.9236.ckpt")][0]
print(f"args_path: {args_path}")
print(f"ckpt_path: {ckpt_path}")

# load arguments
with open(args_path) as f:
    method_args = json.load(f)
print(f"method_args {type(method_args)}: {method_args}")
# build the model
model.pretrained_occlusion_model_dict = None
method_args["backbone"] = model
model_loaded = SupervisedModel_1D.load_from_checkpoint(
    ckpt_path, strict=False, **method_args
)

args_path: trained_models/linear/wxjvsino/args.json
ckpt_path: trained_models/linear/wxjvsino/None-wxjvsino-epoch=21-val_auroc=0.9236.ckpt
method_args <class 'dict'>: {'pretrained_feature_extractor': None, 'validation_frequency': 1, 'dataset': 'ecg-TCH-40_patient-20220201', 'data_dir': None, 'train_dir': None, 'val_dir': None, 'dali': False, 'dali_device': 'gpu', 'no_labels': True, 'debug': False, 'seed': 1, 'ptl_accelerator': 'ddp', 'cluster_name': 'auto', 'read_data_by_chunk': True, 'data_chunk_folder': 'ecg-pat40-tch-sinus_jet', 'channel_ID': 2, 'shift_signal': False, 'normalize_signal': True, 'ecg_resampling_length': 300, 'ecg_resampling_length_target': 300, 'stride': 2, 'c4_multiplier': 3, 'transforms': 'Longitudinal', 'wandb_dir': '/mnt/group1/yilong/JET-Detection-Data/adios', 'logger': True, 'checkpoint_callback': True, 'default_root_dir': None, 'gradient_clip_val': 0.0, 'gradient_clip_algorithm': 'norm', 'process_position': 0, 'num_nodes': 1, 'num_processes': 1, 'devices': None

/Users/yj31/opt/anaconda3/envs/jpy3.7/lib/python3.7/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


In [30]:
model_loaded

SupervisedModel_1D(
  (backbone): cnn_network_contrastive(
    (dropout1): Dropout(p=0.1, inplace=False)
    (dropout2): Dropout(p=0.1, inplace=False)
    (dropout3): Dropout(p=0.1, inplace=False)
    (relu): ReLU()
    (selu): SELU()
    (maxpool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (view_modules): ModuleList(
      (0): Sequential(
        (0): Conv1d(1, 4, kernel_size=(7,), stride=(2,))
        (1): BatchNorm1d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (4): Dropout(p=0.1, inplace=False)
        (5): Conv1d(4, 16, kernel_size=(7,), stride=(2,))
        (6): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (7): ReLU()
        (8): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (9): Dropout(p=0.1, inplace=False)
        (10): Conv

# Run test

In [8]:
feature_with_ecg_df_test_single_lead = data_df
test_dataset = ECG_classification_dataset_with_peak_features(feature_with_ecg_df_test_single_lead,
                                                                     shift_signal=False,
                                                                     shift_amount=0,
                                                                     normalize_signal=True,
                                                                     ecg_resampling_length_target=300)
test_loader = DataLoader(
    test_dataset, batch_size=128, num_workers=1, shuffle=False, drop_last=False
)
for i, data in enumerate(test_loader):
    print(i, len(data))
    break

0 2


In [47]:
X_all_list = []
X_embedding_all_list = []
y_true_all_list = []
X_logits_all_list = []

model_loaded.eval()
test_auroc = AUROC(pos_label=1)
softmax = torch.nn.Softmax(dim=1)

for i, data in enumerate(test_loader):
    print(f"Batch {i} / {len(test_loader)}")
    X, y = data
    X_all_list.append(X)
    X_embedding = model_loaded.backbone(X)
    X_logits = model_loaded.classifier(X_embedding)
#     print(X_embedding.shape, y.shape)
#     print(y)
#     print(X_logits)
    X_embedding_all_list.append(X_embedding)
    y_true_all_list.append(y)
    X_logits_all_list.append(X_logits)
    
    scores = softmax(X_logits)[:, 1].detach()
    test_auroc.update(scores, y.detach())
#     break
    
test_auroc_val = test_auroc.compute()
test_auroc.reset()
print(f"test_auroc_val = {test_auroc_val:.4f}")

/Users/yj31/opt/anaconda3/envs/jpy3.7/lib/python3.7/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


Batch 0 / 894
Batch 1 / 894
Batch 2 / 894
Batch 3 / 894
Batch 4 / 894
Batch 5 / 894
Batch 6 / 894
Batch 7 / 894
Batch 8 / 894
Batch 9 / 894
Batch 10 / 894
Batch 11 / 894
Batch 12 / 894
Batch 13 / 894
Batch 14 / 894
Batch 15 / 894
Batch 16 / 894
Batch 17 / 894
Batch 18 / 894
Batch 19 / 894
Batch 20 / 894
Batch 21 / 894
Batch 22 / 894
Batch 23 / 894
Batch 24 / 894
Batch 25 / 894
Batch 26 / 894
Batch 27 / 894
Batch 28 / 894
Batch 29 / 894
Batch 30 / 894
Batch 31 / 894
Batch 32 / 894
Batch 33 / 894
Batch 34 / 894
Batch 35 / 894
Batch 36 / 894
Batch 37 / 894
Batch 38 / 894
Batch 39 / 894
Batch 40 / 894
Batch 41 / 894
Batch 42 / 894
Batch 43 / 894
Batch 44 / 894
Batch 45 / 894
Batch 46 / 894
Batch 47 / 894
Batch 48 / 894
Batch 49 / 894
Batch 50 / 894
Batch 51 / 894
Batch 52 / 894
Batch 53 / 894
Batch 54 / 894
Batch 55 / 894
Batch 56 / 894
Batch 57 / 894
Batch 58 / 894
Batch 59 / 894
Batch 60 / 894
Batch 61 / 894
Batch 62 / 894
Batch 63 / 894
Batch 64 / 894
Batch 65 / 894
Batch 66 / 894
Batch

Batch 535 / 894
Batch 536 / 894
Batch 537 / 894
Batch 538 / 894
Batch 539 / 894
Batch 540 / 894
Batch 541 / 894
Batch 542 / 894
Batch 543 / 894
Batch 544 / 894
Batch 545 / 894
Batch 546 / 894
Batch 547 / 894
Batch 548 / 894
Batch 549 / 894
Batch 550 / 894
Batch 551 / 894
Batch 552 / 894
Batch 553 / 894
Batch 554 / 894
Batch 555 / 894
Batch 556 / 894
Batch 557 / 894
Batch 558 / 894
Batch 559 / 894
Batch 560 / 894
Batch 561 / 894
Batch 562 / 894
Batch 563 / 894
Batch 564 / 894
Batch 565 / 894
Batch 566 / 894
Batch 567 / 894
Batch 568 / 894
Batch 569 / 894
Batch 570 / 894
Batch 571 / 894
Batch 572 / 894
Batch 573 / 894
Batch 574 / 894
Batch 575 / 894
Batch 576 / 894
Batch 577 / 894
Batch 578 / 894
Batch 579 / 894
Batch 580 / 894
Batch 581 / 894
Batch 582 / 894
Batch 583 / 894
Batch 584 / 894
Batch 585 / 894
Batch 586 / 894
Batch 587 / 894
Batch 588 / 894
Batch 589 / 894
Batch 590 / 894
Batch 591 / 894
Batch 592 / 894
Batch 593 / 894
Batch 594 / 894
Batch 595 / 894
Batch 596 / 894
Batch 59

In [49]:
X_all = torch.cat(X_all_list, dim=0)
X_all.shape

torch.Size([114412, 1, 300])

In [57]:
y_true_all = torch.cat(y_true_all_list, dim=0)
y_true_all.shape

torch.Size([114412])

In [39]:
X_embedding_all = torch.cat(X_embedding_all_list, dim=0)
X_embedding_all.shape

torch.Size([114412, 16])

In [45]:
X_embedding_all_sorted_dict = {}
X_embedding_all_sinds_dict = {}
embedding_idx = 0
_, X_embedding_all_sinds_dict[embedding_idx] \
    = torch.sort(X_embedding_all[:, embedding_idx])
X_embedding_all_sorted_dict[embedding_idx] = X_embedding_all[X_embedding_all_sinds_dict[embedding_idx]]

In [58]:
X_all_sorted = X_all[X_embedding_all_sinds_dict[embedding_idx]]
y_all_sorted = y_true_all[X_embedding_all_sinds_dict[embedding_idx]]

In [73]:
img_folder = f"ecg_sorted_by_embedding{embedding_idx}"
os.makedirs(os.path.join(ckpt_dir, img_folder), exist_ok=True)
label_dict = {0: "SINUS", 1: "JET"}
color_dict = {0: "#0000FF", 1: "#FF0000"}
st = time.time()
show_int = len(X_all_sorted) // 100
for j, ecg in enumerate(X_all_sorted):
    if j % show_int == 0:
        print(f"{j} / {len(X_all_sorted)}")
        name = f"em{embedding_idx} ecg{j}"
        y_label = int(y_all_sorted[j].numpy())
        plt.figure(figsize=(6, 3))
        plt.plot(ecg.ravel(), label=f"{name} - {label_dict[y_label]}", color=color_dict[y_label])
        plt.legend(loc="upper center")
        plt.savefig(os.path.join(ckpt_dir, img_folder, f"{name}.png"), dpi=25)
        plt.close()
#     if j >= 5:
#         break

0 / 114412
1144 / 114412
2288 / 114412
3432 / 114412
4576 / 114412
5720 / 114412
6864 / 114412
8008 / 114412
9152 / 114412
10296 / 114412
11440 / 114412
12584 / 114412
13728 / 114412
14872 / 114412
16016 / 114412
17160 / 114412
18304 / 114412
19448 / 114412
20592 / 114412
21736 / 114412
22880 / 114412
24024 / 114412
25168 / 114412
26312 / 114412
27456 / 114412
28600 / 114412
29744 / 114412
30888 / 114412
32032 / 114412
33176 / 114412
34320 / 114412
35464 / 114412
36608 / 114412
37752 / 114412
38896 / 114412
40040 / 114412
41184 / 114412
42328 / 114412
43472 / 114412
44616 / 114412
45760 / 114412
46904 / 114412
48048 / 114412
49192 / 114412
50336 / 114412
51480 / 114412
52624 / 114412
53768 / 114412
54912 / 114412
56056 / 114412
57200 / 114412
58344 / 114412
59488 / 114412
60632 / 114412
61776 / 114412
62920 / 114412
64064 / 114412
65208 / 114412
66352 / 114412
67496 / 114412
68640 / 114412
69784 / 114412
70928 / 114412
72072 / 114412
73216 / 114412
74360 / 114412
75504 / 114412
76648 /

In [74]:
for embedding_idx in range(X_embedding_all.shape[1]):
    # embedding_idx = 0
    _, X_embedding_all_sinds_dict[embedding_idx] \
        = torch.sort(X_embedding_all[:, embedding_idx])
    X_embedding_all_sorted_dict[embedding_idx] = X_embedding_all[X_embedding_all_sinds_dict[embedding_idx]]
    X_all_sorted = X_all[X_embedding_all_sinds_dict[embedding_idx]]
    y_all_sorted = y_true_all[X_embedding_all_sinds_dict[embedding_idx]]
    img_folder = f"ecg_sorted_by_embedding{embedding_idx}"
    os.makedirs(os.path.join(ckpt_dir, img_folder), exist_ok=True)
    label_dict = {0: "SINUS", 1: "JET"}
    color_dict = {0: "#0000FF", 1: "#FF0000"}
    st = time.time()
    show_int = len(X_all_sorted) // 100
    for j, ecg in enumerate(X_all_sorted):
        if j % show_int == 0:
            print(f"{embedding_idx} / {X_embedding_all.shape[1]}, {j} / {len(X_all_sorted)}")
            name = f"em{embedding_idx} ecg{j}"
            y_label = int(y_all_sorted[j].numpy())
            plt.figure(figsize=(6, 3))
            plt.plot(ecg.ravel(), label=f"{name} - {label_dict[y_label]}", color=color_dict[y_label])
            plt.legend(loc="upper center")
            plt.savefig(os.path.join(ckpt_dir, img_folder, f"{name}.png"), dpi=25)
            plt.close()
#     if j >= 5:
#         break

0 / 16, 0 / 114412
0 / 16, 1144 / 114412
0 / 16, 2288 / 114412
0 / 16, 3432 / 114412
0 / 16, 4576 / 114412
0 / 16, 5720 / 114412
0 / 16, 6864 / 114412
0 / 16, 8008 / 114412
0 / 16, 9152 / 114412
0 / 16, 10296 / 114412
0 / 16, 11440 / 114412
0 / 16, 12584 / 114412
0 / 16, 13728 / 114412
0 / 16, 14872 / 114412
0 / 16, 16016 / 114412
0 / 16, 17160 / 114412
0 / 16, 18304 / 114412
0 / 16, 19448 / 114412
0 / 16, 20592 / 114412
0 / 16, 21736 / 114412
0 / 16, 22880 / 114412
0 / 16, 24024 / 114412
0 / 16, 25168 / 114412
0 / 16, 26312 / 114412
0 / 16, 27456 / 114412
0 / 16, 28600 / 114412
0 / 16, 29744 / 114412
0 / 16, 30888 / 114412
0 / 16, 32032 / 114412
0 / 16, 33176 / 114412
0 / 16, 34320 / 114412
0 / 16, 35464 / 114412
0 / 16, 36608 / 114412
0 / 16, 37752 / 114412
0 / 16, 38896 / 114412
0 / 16, 40040 / 114412
0 / 16, 41184 / 114412
0 / 16, 42328 / 114412
0 / 16, 43472 / 114412
0 / 16, 44616 / 114412
0 / 16, 45760 / 114412
0 / 16, 46904 / 114412
0 / 16, 48048 / 114412
0 / 16, 49192 / 114412


3 / 16, 64064 / 114412
3 / 16, 65208 / 114412
3 / 16, 66352 / 114412
3 / 16, 67496 / 114412
3 / 16, 68640 / 114412
3 / 16, 69784 / 114412
3 / 16, 70928 / 114412
3 / 16, 72072 / 114412
3 / 16, 73216 / 114412
3 / 16, 74360 / 114412
3 / 16, 75504 / 114412
3 / 16, 76648 / 114412
3 / 16, 77792 / 114412
3 / 16, 78936 / 114412
3 / 16, 80080 / 114412
3 / 16, 81224 / 114412
3 / 16, 82368 / 114412
3 / 16, 83512 / 114412
3 / 16, 84656 / 114412
3 / 16, 85800 / 114412
3 / 16, 86944 / 114412
3 / 16, 88088 / 114412
3 / 16, 89232 / 114412
3 / 16, 90376 / 114412
3 / 16, 91520 / 114412
3 / 16, 92664 / 114412
3 / 16, 93808 / 114412
3 / 16, 94952 / 114412
3 / 16, 96096 / 114412
3 / 16, 97240 / 114412
3 / 16, 98384 / 114412
3 / 16, 99528 / 114412
3 / 16, 100672 / 114412
3 / 16, 101816 / 114412
3 / 16, 102960 / 114412
3 / 16, 104104 / 114412
3 / 16, 105248 / 114412
3 / 16, 106392 / 114412
3 / 16, 107536 / 114412
3 / 16, 108680 / 114412
3 / 16, 109824 / 114412
3 / 16, 110968 / 114412
3 / 16, 112112 / 114412


7 / 16, 10296 / 114412
7 / 16, 11440 / 114412
7 / 16, 12584 / 114412
7 / 16, 13728 / 114412
7 / 16, 14872 / 114412
7 / 16, 16016 / 114412
7 / 16, 17160 / 114412
7 / 16, 18304 / 114412
7 / 16, 19448 / 114412
7 / 16, 20592 / 114412
7 / 16, 21736 / 114412
7 / 16, 22880 / 114412
7 / 16, 24024 / 114412
7 / 16, 25168 / 114412
7 / 16, 26312 / 114412
7 / 16, 27456 / 114412
7 / 16, 28600 / 114412
7 / 16, 29744 / 114412
7 / 16, 30888 / 114412
7 / 16, 32032 / 114412
7 / 16, 33176 / 114412
7 / 16, 34320 / 114412
7 / 16, 35464 / 114412
7 / 16, 36608 / 114412
7 / 16, 37752 / 114412
7 / 16, 38896 / 114412
7 / 16, 40040 / 114412
7 / 16, 41184 / 114412
7 / 16, 42328 / 114412
7 / 16, 43472 / 114412
7 / 16, 44616 / 114412
7 / 16, 45760 / 114412
7 / 16, 46904 / 114412
7 / 16, 48048 / 114412
7 / 16, 49192 / 114412
7 / 16, 50336 / 114412
7 / 16, 51480 / 114412
7 / 16, 52624 / 114412
7 / 16, 53768 / 114412
7 / 16, 54912 / 114412
7 / 16, 56056 / 114412
7 / 16, 57200 / 114412
7 / 16, 58344 / 114412
7 / 16, 594

10 / 16, 68640 / 114412
10 / 16, 69784 / 114412
10 / 16, 70928 / 114412
10 / 16, 72072 / 114412
10 / 16, 73216 / 114412
10 / 16, 74360 / 114412
10 / 16, 75504 / 114412
10 / 16, 76648 / 114412
10 / 16, 77792 / 114412
10 / 16, 78936 / 114412
10 / 16, 80080 / 114412
10 / 16, 81224 / 114412
10 / 16, 82368 / 114412
10 / 16, 83512 / 114412
10 / 16, 84656 / 114412
10 / 16, 85800 / 114412
10 / 16, 86944 / 114412
10 / 16, 88088 / 114412
10 / 16, 89232 / 114412
10 / 16, 90376 / 114412
10 / 16, 91520 / 114412
10 / 16, 92664 / 114412
10 / 16, 93808 / 114412
10 / 16, 94952 / 114412
10 / 16, 96096 / 114412
10 / 16, 97240 / 114412
10 / 16, 98384 / 114412
10 / 16, 99528 / 114412
10 / 16, 100672 / 114412
10 / 16, 101816 / 114412
10 / 16, 102960 / 114412
10 / 16, 104104 / 114412
10 / 16, 105248 / 114412
10 / 16, 106392 / 114412
10 / 16, 107536 / 114412
10 / 16, 108680 / 114412
10 / 16, 109824 / 114412
10 / 16, 110968 / 114412
10 / 16, 112112 / 114412
10 / 16, 113256 / 114412
10 / 16, 114400 / 114412
11 

13 / 16, 114400 / 114412
14 / 16, 0 / 114412
14 / 16, 1144 / 114412
14 / 16, 2288 / 114412
14 / 16, 3432 / 114412
14 / 16, 4576 / 114412
14 / 16, 5720 / 114412
14 / 16, 6864 / 114412
14 / 16, 8008 / 114412
14 / 16, 9152 / 114412
14 / 16, 10296 / 114412
14 / 16, 11440 / 114412
14 / 16, 12584 / 114412
14 / 16, 13728 / 114412
14 / 16, 14872 / 114412
14 / 16, 16016 / 114412
14 / 16, 17160 / 114412
14 / 16, 18304 / 114412
14 / 16, 19448 / 114412
14 / 16, 20592 / 114412
14 / 16, 21736 / 114412
14 / 16, 22880 / 114412
14 / 16, 24024 / 114412
14 / 16, 25168 / 114412
14 / 16, 26312 / 114412
14 / 16, 27456 / 114412
14 / 16, 28600 / 114412
14 / 16, 29744 / 114412
14 / 16, 30888 / 114412
14 / 16, 32032 / 114412
14 / 16, 33176 / 114412
14 / 16, 34320 / 114412
14 / 16, 35464 / 114412
14 / 16, 36608 / 114412
14 / 16, 37752 / 114412
14 / 16, 38896 / 114412
14 / 16, 40040 / 114412
14 / 16, 41184 / 114412
14 / 16, 42328 / 114412
14 / 16, 43472 / 114412
14 / 16, 44616 / 114412
14 / 16, 45760 / 114412
14 

In [ ]:
for embedding_idx in range(X_embedding_all.shape[1]):
    _, X_embedding_all_sinds_dict[embedding_idx] \
        = torch.sort(X_embedding_all[:, embedding_idx])
    X_embedding_all_sorted_dict[embedding_idx] = X_embedding_all[X_embedding_all_sinds_dict[embedding_idx]]

In [ ]:
for embedding_idx in range(X_embedding_all.shape[1]):
    # embedding_idx = 0
    X_all_sorted = X_all[X_embedding_all_sinds_dict[embedding_idx]]
    y_all_sorted = y_true_all[X_embedding_all_sinds_dict[embedding_idx]]
    img_folder = f"ecg_sorted_by_embedding{embedding_idx}"
    os.makedirs(os.path.join(ckpt_dir, img_folder), exist_ok=True)
    label_dict = {0: "SINUS", 1: "JET"}
    color_dict = {0: "#0000FF", 1: "#FF0000"}
    st = time.time()
    show_int = len(X_all_sorted) // 100
    for j, ecg in enumerate(X_all_sorted):
        if j % show_int == 0:
            print(f"{embedding_idx} / {X_embedding_all.shape[1]}, {j} / {len(X_all_sorted)}")
            name = f"em{embedding_idx} ecg{j}"
            y_label = int(y_all_sorted[j].numpy())
            plt.figure(figsize=(6, 3))
            plt.plot(ecg.ravel(), label=f"{name} - {label_dict[y_label]}", color=color_dict[y_label])
            plt.legend(loc="upper center")
            plt.savefig(os.path.join(ckpt_dir, img_folder, f"{name}.png"), dpi=25)
            plt.close()
#     if j >= 5:
#         break